In [48]:
from bs4 import BeautifulSoup
import warnings
import requests
import pandas as pd

In [49]:
warnings.filterwarnings("ignore", category=UserWarning, module='bs4')                  #To clear warning message 
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
html = requests.get(url)
#html.headers
c = html.content
soup = BeautifulSoup(c)
print(soup.prettify())
My_table = soup.find('table',{'class':'wikitable sortable'}) 


<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of postal codes of Canada: M - Wikipedia
  </title>
  <script>
   document.documentElement.className=document.documentElement.className.replace(/(^|\s)client-nojs(\s|$)/,"$1client-js$2");RLCONF={"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":906439794,"wgRevisionId":906439794,"wgArticleId":539066,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Communications in Ontario","Postal codes in Canada","Toronto","Ontario-related lists"],"wgBreakFrames":!1,"wgPageContentLanguage":"en","wgPageContentModel":"wikitext","wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June",

In [50]:
links = My_table.findAll('tr')           #tr tag contains all the info of a neighbourhood 
row = len(links)
headers = links[0].text.split()          #the first row[0] contains headers of the table 
headers

['Postcode', 'Borough', 'Neighbourhood']

In [51]:
#Looping through the table and storing data in a pandas data frame

records =[]                                             
n=1
while n < row :
    Postcode=links[n].text.split('\n')[1]
    Borough=links[n].text.split('\n')[2]
    Neighborhood=links[n].text.split('\n')[3]
    records.append((Postcode, Borough,Neighborhood))
    n=n+1

df=pd.DataFrame(records, columns=['PostalCode', 'Borough', 'Neighbourhood'])
df.head(5)

,PostalCode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [52]:
df[df['Borough']=='Not assigned'].count()

PostalCode       77
Borough          77
Neighbourhood    77
dtype: int64

In [53]:
# Droping "not assigned" values from coloumn 'Borough' and replacing "not assigned" values of 'Neighbourhood' with corresponding 'Borough' value

df1=df[~df.Borough.str.contains("Not assigned")]
df1=df1.reset_index(drop=True)
df1.head(20)
df1.loc[df1['Neighbourhood'] == 'Not assigned'] = df1['Borough']
df1.head(20)

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,Queen's Park,Queen's Park,Queen's Park
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


In [54]:
#checking no. of unique values of each column 
postalcodes = df1['PostalCode'].nunique()
boroughs = df1['Borough'].nunique()
neighbourhoods= df1['Neighbourhood'].nunique()
print(postalcodes,boroughs,neighbourhoods,len(df1))

103 11 209 211


In [55]:
# concatenating neighbourhoods with the same postal code

n=0
limit = len(df1)-1

while n< limit:
    row1 = df1.iloc[n,0]               #first postal code in the dataframe
    m=n+1
    row2 = df1.iloc[m,0]               #2nd postal code in the dataframe
    neigh1 = df1.iloc[n,2]             #first neighbourhood
    neigh2 = df1.iloc[m,2]             #2nd neighbourhood 
    
    if row1==row2:                                                   #comparing 1st and 2nd postal code
        df1.Neighbourhood[n,2] = neigh1=neigh1+','+neigh2            #concatenating neighbourhoods if the postal codes are same 
        df1=df1.drop(df1.index[m])                                   #droping duplicate row
        limit=limit-1                                                #decrementing total no. of rows
        df1 = df1.reset_index(drop=True)
    else:
        n=n+1
        
df1.sort_values('PostalCode').head(20)        
        
        

,PostalCode,Borough,Neighbourhood
6,M1B,Scarborough,"Rouge,Malvern"
12,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
18,M1E,Scarborough,"Guildwood,Morningside,West Hill"
22,M1G,Scarborough,Woburn
26,M1H,Scarborough,Cedarbrae
32,M1J,Scarborough,Scarborough Village
38,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park"
44,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge"
51,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West"
58,M1N,Scarborough,"Birch Cliff,Cliffside West"


In [56]:
df1.shape

(103, 3)

PART 2
cONCATENATING LATITUDES AND LONGITUDES OF THE NEIGHBOURHOODS WITH THE DATASET


In [57]:
!wget -q -O "toronto_coordinates.csv" http://cocl.us/Geospatial_data
cordinates = pd.read_csv('toronto_coordinates.csv')
#cordinates

index1 = df1.set_index('PostalCode')
index2= cordinates.set_index('Postal Code')
Data = pd.concat([index1,index2],axis=1,join='inner')
#Data
Data.index.name = 'PostalCode'
Data.reset_index(inplace=True)
Data.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Kingsway Park South West,Mimico NW,The Queensw...",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
4,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242


PART 3 CLustering

In [58]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(Data['Borough'].unique()),
       Data.shape[0]
    )
)

The dataframe has 10 boroughs and 102 neighborhoods.


In [59]:
from geopy.geocoders import Nominatim 
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium
#!conda install -c conda-forge folium=0.5.0 --yes


In [60]:
address="Toronto, TR"
geolocator = Nominatim(user_agent="my_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are -33.013441, 151.5942043.


In [66]:
#Creating a map of Toronto city with neighbourhoods superimposed
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, long, post, borough, neigh in zip(Data['Latitude'], Data['Longitude'], Data['PostalCode'], Data['Borough'], Data['Neighbourhood']):
    label = "{} ({}): {}".format(borough, post, neigh)
    popup = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, long],
        radius=5,
        popup=popup,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)
    
map_toronto